# Yelp API - Lab

The previously deployed lab on working around building a GIS with Yelp API and Folium can be found [here](https://github.com/learn-co-curriculum/dsc-2-15-10-yelp-api-gis-lab/tree/a56358c2d0c2daf569a5f50937c4c27463aadb1a) (not relevant for new students).


## Introduction 

Now that we've seen how the Yelp API works, and some basic Folium visualizations its time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [ ]:
#Your code here

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.josn().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [2]:
import time
import requests

client_id = 'eAOaimX85y-R9FkFMWxzPQ'
api_key = 'xrzz0vUQAvouoVJLGB_djsWy6yNaV4VIQldPJx7ssV0_zCWtODVW00DFenmlYBJoNqcxBQ7YJoldbggFIXW4C0BiwgcqmM1DowOhH1SRmnyNLZW4BadfqbMAlIVcXHYx'

# print(response.json().keys())
term = 'bike'
location = 'Minneapolis'
SEARCH_LIMIT = 50
offset = 0


url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT,
                'offset': offset

            }



mpls_bike = []

for i in range(0,4):
    print(offset)
    url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT,
                'offset': offset

            }
    print(url_params)
    response = requests.get(url, headers=headers, params=url_params)
    for x in range(len(response.json()['businesses'])):
        mpls_bike.append(response.json()['businesses'][x])
    offset +=50

0
{'term': 'bike', 'location': 'Minneapolis', 'limit': 50, 'offset': 0}
50
{'term': 'bike', 'location': 'Minneapolis', 'limit': 50, 'offset': 50}
100
{'term': 'bike', 'location': 'Minneapolis', 'limit': 50, 'offset': 100}
150
{'term': 'bike', 'location': 'Minneapolis', 'limit': 50, 'offset': 150}


## Exploratory Analysis

Take the restaurants from the previous question and do an intial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [3]:
parsed_mpls_list = []
closed_bike = []

def clean_yelp(lst):
    for x in range(len(mpls_bike)):
        parsed = {}
        parsed['name'] = mpls_bike[x]['name']
        parsed['review_count'] = mpls_bike[x]['review_count']
        parsed['coordinates'] = mpls_bike[x]['coordinates']
        if 'price' in mpls_bike[x]:
            parsed['price'] = mpls_bike[x]['price']
            
        parsed_mpls_list.append(parsed) 
    
clean_yelp(mpls_bike)
print(parsed_mpls_list[1])

{'name': 'The Hub Bike Co-op', 'review_count': 63, 'coordinates': {'latitude': 44.9477389, 'longitude': -93.2342847}, 'price': '$$'}


## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [5]:
!pip install folium



    100% |████████████████████████████████| 92kB 18.4MB/s ta 0:00:01
You are using pip version 10.0.1, however version 19.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
import os
import folium
import pandas as pd

In [8]:
df = pd.DataFrame.from_dict(mpls_bike)

In [12]:
df

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,two-wheels-bike-shop-minneapolis,"[{'alias': 'bikes', 'title': 'Bikes'}, {'alias...","{'latitude': 44.9538248, 'longitude': -93.2927...",(612) 735-7345,1320.845397,zRfiz_X2fUMgI_5FspHCNA,https://s3-media3.fl.yelpcdn.com/bphoto/rGKfDZ...,False,"{'address1': '1014 W 27th St', 'address2': '',...",Two Wheels Bike Shop,+16127357345,$$,5.0,15,[],https://www.yelp.com/biz/two-wheels-bike-shop-...
1,the-hub-bike-co-op-minneapolis-2,"[{'alias': 'bikes', 'title': 'Bikes'}, {'alias...","{'latitude': 44.9477389, 'longitude': -93.2342...",(612) 729-0437,4841.259513,DsfE5iuN8CVTXwKp9GeDLw,https://s3-media3.fl.yelpcdn.com/bphoto/Q-osbc...,False,"{'address1': '3016 Minnehaha Ave', 'address2':...",The Hub Bike Co-op,+16127290437,$$,4.5,63,[],https://www.yelp.com/biz/the-hub-bike-co-op-mi...
2,alternative-bike-and-board-shop-minneapolis,"[{'alias': 'bikes', 'title': 'Bikes'}]","{'latitude': 44.9478416442871, 'longitude': -9...",(612) 374-3635,1984.301923,UOFoLzpGmLhf1FUTOHJfOg,https://s3-media3.fl.yelpcdn.com/bphoto/7KTBxb...,False,"{'address1': '3013 Lyndale Ave S', 'address2':...",Alternative Bike & Board Shop,+16123743635,$$,4.5,42,[],https://www.yelp.com/biz/alternative-bike-and-...
3,tangletown-bike-shop-minneapolis-2,"[{'alias': 'bikes', 'title': 'Bikes'}, {'alias...","{'latitude': 44.9125299, 'longitude': -93.29043}",(612) 259-8180,5906.520137,JD1QDjk1Tbu0jP-MR9sBHA,https://s3-media4.fl.yelpcdn.com/bphoto/6dgzaQ...,False,"{'address1': '816 W 50th St', 'address2': '', ...",Tangletown Bike Shop,+16122598180,$$,5.0,17,[],https://www.yelp.com/biz/tangletown-bike-shop-...
4,recovery-bike-shop-minneapolis,"[{'alias': 'bikes', 'title': 'Bikes'}, {'alias...","{'latitude': 45.01342, 'longitude': -93.24787}",(612) 876-5356,6296.830889,wuA-LUJ1juuM6WX5-JWJzA,https://s3-media4.fl.yelpcdn.com/bphoto/XQEf1J...,False,"{'address1': '2504 Central Ave NE', 'address2'...",Recovery Bike Shop,+16128765356,$$,4.5,29,[],https://www.yelp.com/biz/recovery-bike-shop-mi...
5,penn-cycle-minneapolis,"[{'alias': 'bikes', 'title': 'Bikes'}]","{'latitude': 44.9485283, 'longitude': -93.2886...",(612) 822-2228,1902.286181,HTJVGFTa21NiCPViDv3f8g,https://s3-media2.fl.yelpcdn.com/bphoto/nYrM1f...,False,"{'address1': '710 W Lake St', 'address2': '', ...",Penn Cycle,+16128222228,$$,4.0,27,[],https://www.yelp.com/biz/penn-cycle-minneapoli...
6,freewheel-bicycle-minneapolis,"[{'alias': 'bikes', 'title': 'Bikes'}]","{'latitude': 44.9680899, 'longitude': -93.24648}",(612) 339-2219,3472.838092,Yyh8v4r81ODd6IX5OxYi-w,https://s3-media2.fl.yelpcdn.com/bphoto/MHjQyu...,False,"{'address1': '1812 S 6th St', 'address2': '', ...",Freewheel Bicycle,+16123392219,$$,4.0,28,[],https://www.yelp.com/biz/freewheel-bicycle-min...
7,perennial-cycle-minneapolis,"[{'alias': 'bikes', 'title': 'Bikes'}]","{'latitude': 44.94159, 'longitude': -93.29871}",(612) 827-8000,2744.887871,hlaELWyOwL9hfCimeI1F7Q,https://s3-media2.fl.yelpcdn.com/bphoto/uBand3...,False,"{'address1': '3342 Hennepin Ave', 'address2': ...",Perennial Cycle,+16128278000,$$,4.0,30,[],https://www.yelp.com/biz/perennial-cycle-minne...
8,one-on-one-bicycle-studio-minneapolis,"[{'alias': 'bikes', 'title': 'Bikes'}, {'alias...","{'latitude': 44.98311, 'longitude': -93.27125}",(612) 371-9565,2467.249020,trN0U4LVzQeVzohQ0dIBuQ,https://s3-media2.fl.yelpcdn.com/bphoto/pGQ7f3...,False,"{'address1': '117 Washington Ave N', 'address2...",One On One Bicycle Studio,+16123719565,$$,4.0,79,[],https://www.yelp.com/biz/one-on-one-bicycle-st...
9,eriks-bike-board-ski-minneapolis-6,"[{'alias': 'bikes', 'title': 'Bikes'}, {'alias...","{'latitude': 44.95159, 'longitude': -93.2885}",(612) 872-8600,1572.010349,hTbvHB1mEMLiC9Y3F2KCuA,https://s3-media3.fl.yelpcdn.com/bphoto/SyndjC...,False,"{'address1': '2800 Lyndale Ave S', 'address2':...",ERIK'S Bike Board Ski,+16128728600,$$,5.0,5,[],https://www.yelp.com/biz/eriks-bike-bo

In [17]:
mpls_coord = (44.9778, -93.2650)

m = folium.Map(location=mpls_coord, zoom_start=12)

for i in range(0,len(df)):
    folium.Marker([df.iloc[i]['coordinates'], popup =df.iloc[i]['name']).add_to(m)


SyntaxError: invalid syntax (<ipython-input-17-ae786cb30603>, line 6)

In [15]:
df.iloc[4]['coordinates']

{'latitude': 45.01342, 'longitude': -93.24787}

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!